# <font color='violet'> Complete Feature Engineering & Assign Ratigns to Psychedelic Experience Reports
    
Drawing from scraped psychedelic experience reports that were cleaned here: https://github.com/fractaldatalearning/psychedelic_efficacy/blob/main/notebooks/9-kl-reports-clean-engineer.ipynb

In [ ]:
import

In [ ]:
# prepare to add local python functions; import modules from src directory
import sys
src = '../src'
sys.path.append(src)

# import local functions
from nlp.feature_engineer import analyze_sentiment

In [ ]:
df = pd.read_csv('../data/processed/erowid_cleaned.csv')
df.info()

<font color='violet'> Next Thing. 

In [ ]:
df.to_csv('../data/processed/name_this_data.csv')